In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -U datasets

In [20]:
import os
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.cuda.amp import autocast, GradScaler
import json
import re
import random
from tqdm import tqdm
from collections import Counter, defaultdict
from datasets import load_dataset
import unicodedata
import math
from pathlib import Path

In [2]:
# Device configuration
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Set seed for reproducibility
torch.manual_seed(1337)

Using device: cuda


In [3]:
# Create checkpoint directories
os.makedirs("checkpoints", exist_ok=True)
os.makedirs("checkpoints/best_model", exist_ok=True)

# Data loading and preprocessing
def clean_text(text: str) -> str:
    """Türkçe metni temizle ve normalize et."""
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(r'\[.*?\]|\(.*?\)', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def load_and_preprocess_data(max_samples=50000):
    """Veri setini yükleyip temizler, summary listesini döner."""
    dataset = load_dataset("musabg/wikipedia-tr-summarization", split='train')
    processed_texts = []

    for i in tqdm(range(min(len(dataset), max_samples)), desc="Preprocessing data"):
        summary = clean_text(dataset[i]["summary"])
        processed_texts.append(summary)

    return processed_texts

In [4]:
import re
import json
import random
from collections import Counter, defaultdict
from tqdm import tqdm
from typing import List, Dict, Tuple, Optional

class OptimizedByteLevelBPE:
    def __init__(self, merges: Optional[List[Tuple[str, str]]] = None,
                 vocab: Optional[Dict[str, int]] = None,
                 special_tokens: Optional[List[str]] = None):
        self.merges = merges
        self.vocab = vocab or {}
        self.special_tokens = special_tokens or ['<pad>', '<unk>', '<sos>', '<eos>']
        self._build_lookup_tables()

    def _build_lookup_tables(self):
        self.token_to_id = {}
        self.id_to_token = {}

        # Özel token'lar
        for idx, token in enumerate(self.special_tokens):
            self.token_to_id[token] = idx

        offset = len(self.token_to_id)

        # Byte token'lar
        for i in range(256):
            byte_token = f"{i:03d}"
            self.token_to_id[byte_token] = offset + i

        offset = max(self.token_to_id.values()) + 1

        # Eğer vocab varsa, ekle
        if self.vocab:
            for token in sorted(self.vocab.keys()):
                if token not in self.token_to_id:
                    self.token_to_id[token] = offset
                    offset += 1

        self.id_to_token = {v: k for k, v in self.token_to_id.items()}
        self.special_token_ids = {tok: self.token_to_id[tok] for tok in self.special_tokens}
        self.merges_set = set(self.merges)


    def _build_token_vocab(self):
        """Merge sonrası oluşan token sözlüğünü oluşturur"""
        tokens = set()
        for a, b in self.merges:
            tokens.add(a)
            tokens.add(b)
            tokens.add(a + b)
        tokens = sorted(tokens)

        # Token ID'lerini devam ettir
        start_id = max(self.token_to_id.values()) + 1
        for tok in tokens:
            if tok not in self.token_to_id:
                self.token_to_id[tok] = start_id
                start_id += 1

        self.id_to_token = {v: k for k, v in self.token_to_id.items()}
        self.vocab = {k: v for k, v in self.token_to_id.items() if k not in self.special_tokens}

    def train(self, corpus: List[str], num_merges: int = 10000,
              chunk_size: int = 10000, verbose: bool = True):
        """BPE algoritması ile tokenizer'ı eğitir"""
        global_freqs = Counter()

        # 1. Frekansları hesapla
        for i in tqdm(range(0, len(corpus), chunk_size), desc="Vocabulary Construction"):
            chunk = corpus[i:i + chunk_size]
            text = " ".join(chunk)
            words = re.findall(r'\w+|[^\w\s]|\s+', text)

            for word in words:
                byte_tokens = [f"{b:03d}" for b in word.encode("utf-8")] + ["</w>"]
                global_freqs[" ".join(byte_tokens)] += 1

        # 2. Merge işlemleri
        vocab = global_freqs
        self.merges = []

        for merge_step in tqdm(range(num_merges), desc="BPE Merging"):
            pairs = self._get_stats(vocab)
            if not pairs:
                break

            best_pair = max(pairs.items(), key=lambda x: x[1])[0]
            vocab = self._merge_vocab(best_pair, vocab)
            self.merges.append(best_pair)

            if verbose and (merge_step % 1000 == 0 or merge_step == num_merges - 1):
                print(f"Merge {merge_step + 1}: {best_pair} (freq: {pairs[best_pair]})")

        self.merges_set = set(self.merges)
        self._build_token_vocab()
        self._build_lookup_tables()

    def _get_stats(self, vocab: Counter) -> Dict[Tuple[str, str], int]:
        """Sembollerin eş frekanslarını hesaplar"""
        pairs = defaultdict(int)
        for word, freq in vocab.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pairs[(symbols[i], symbols[i + 1])] += freq
        return pairs

    def _merge_vocab(self, pair: Tuple[str, str], vocab: Counter) -> Counter:
        """En sık geçen çifti birleştirir"""
        new_vocab = Counter()
        pattern = re.compile(rf'(?<!\S){re.escape(pair[0])} {re.escape(pair[1])}(?!\S)')

        for word, freq in vocab.items():
            new_word = pattern.sub(pair[0] + pair[1], word)
            new_vocab[new_word] = freq

        return new_vocab

    def encode(self, text: str, dropout: float = 0.0) -> List[int]:
        """Metni token ID'lerine çevirir"""
        words = re.findall(r'\w+|[^\w\s]|\s+', text)
        token_ids = []

        for word in words:
            tokens = [f"{b:03d}" for b in word.encode("utf-8")]

            # BPE merge
            while len(tokens) > 1:
                pairs = [(tokens[i], tokens[i + 1]) for i in range(len(tokens) - 1)]
                valid_pairs = [
                    p for p in pairs
                    if p in self.merges_set and random.random() > dropout
                ]
                if not valid_pairs:
                    break

                best_pair = min(valid_pairs, key=lambda p: self.merges.index(p))
                merged_token = best_pair[0] + best_pair[1]

                new_tokens = []
                i = 0
                while i < len(tokens):
                    if i < len(tokens) - 1 and (tokens[i], tokens[i + 1]) == best_pair:
                        new_tokens.append(merged_token)
                        i += 2
                    else:
                        new_tokens.append(tokens[i])
                        i += 1

                tokens = new_tokens

            for token in tokens:
                token_ids.append(self.token_to_id.get(token, self.special_token_ids["<unk>"]))

        return token_ids

    def decode(self, token_ids: List[int]) -> str:
        """Token ID'lerinden orijinal metni oluşturur"""
        tokens = [self.id_to_token.get(tid, '<unk>') for tid in token_ids]
        decoded_bytes = []

        for token in tokens:
            if token in self.special_token_ids:
                continue  # özel token'ları atla
            try:
                # Tüm token'ı 3'er 3'er parçala
                bytes_seq = [int(token[i:i+3]) for i in range(0, len(token), 3)]
                decoded_bytes.extend(bytes_seq)
            except ValueError:
                pass  # bilinmeyen token varsa yoksay

        try:
            return bytes(decoded_bytes).decode('utf-8', errors='replace')
        except Exception:
            return "Corrupted"



    def save_model(self, prefix: str):
        """Modeli diske kaydeder"""
        with open(prefix, "w", encoding="utf-8") as f:
            json.dump({
                "merges": self.merges,
                "vocab": self.vocab,
                "special_tokens": self.special_tokens
            }, f, ensure_ascii=False)

    @classmethod
    def load_model(cls, prefix: str):
        with open(prefix, "r", encoding="utf-8") as f:
            data = json.load(f)
        merges = [tuple(m) for m in data["merges"]]
        obj = cls(
            merges=merges,
            vocab=data["vocab"],
            special_tokens=data["special_tokens"]
        )
        # Eksik yapılandırmaları tamamla
        obj._build_token_vocab()
        obj._build_lookup_tables()
        return obj



In [5]:
# Test için tokenizer'ı kontrol edin
tokenizer = OptimizedByteLevelBPE.load_model("tokenizer.json")
test_text = "Çin'de yapılan bir araştırmaya göre"
encoded = tokenizer.encode(test_text)
decoded = tokenizer.decode(encoded)
print("Encoded:", encoded)
print("Decoded:", decoded)  # Orijinal metni tam olarak geri almalı

Encoded: [2944, 1822, 43, 104, 105, 36, 2848, 988, 36, 1180, 36, 1031, 2214, 101, 36, 1654, 105]
Decoded: Çin'de yapılan bir araştırmaya göre


In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import json
from safetensors.torch import save_model, load_model
import os

class Head(nn.Module):
    def __init__(self, head_size, n_embd, block_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * (C ** -0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size, n_embd, block_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size, n_embd, block_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 2 * n_embd),
            nn.ReLU(),
            nn.Linear(2 * n_embd, n_embd),
            nn.Dropout(0.1),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head, block_size):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size, n_embd, block_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class Transformer(nn.Module):
    def __init__(self, vocab_size, n_embd=512, block_size=256, n_layer=6, n_head=8, tokenizer=None, device='cuda'):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head, block_size) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.block_size = block_size
        self.tokenizer = tokenizer
        self.device = device

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=self.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        loss = None
        if targets is not None:
            logits = logits.view(B * T, -1)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens=100, temperature=1.0, top_k=30):
        self.eval()
        with torch.no_grad():
            for _ in range(max_new_tokens):
                idx_cond = idx[:, -self.block_size:]
                logits, _ = self(idx_cond)
                logits = logits[:, -1, :] / temperature

                if top_k is not None:
                    v, _ = torch.topk(logits, top_k)
                    logits[logits < v[:, [-1]]] = -float('Inf')

                probs = F.softmax(logits, dim=-1)
                next_token = torch.multinomial(probs, num_samples=1)
                idx = torch.cat([idx, next_token], dim=1)

        return idx

    def generate_from_prompt(self, prompt, max_new_tokens=100, temperature=1.0, top_k=None):
        self.eval()
        tokens = self.tokenizer.encode(prompt)
        context = torch.tensor(tokens, dtype=torch.long, device=self.device).unsqueeze(0)
        generated = self.generate(context, max_new_tokens=max_new_tokens, temperature=temperature, top_k=top_k)
        return self.tokenizer.decode(generated[0].tolist())

    def save_model(self, filepath, optimizer=None, iter_step=None, best_val_loss=None):
        config = {
            'vocab_size': len(self.tokenizer.token_to_id),
            'n_embd': self.token_embedding_table.embedding_dim,
            'block_size': self.block_size,
            'n_layer': len(self.blocks),
            'n_head': len(self.blocks[0].sa.heads),
            'tokenizer_config': {
                'merges': self.tokenizer.merges,
                'vocab': self.tokenizer.vocab,
                'special_tokens': self.tokenizer.special_tokens
            }
        }

        config_path = filepath.replace('.safetensors', '_config.json')
        trainstate_path = filepath.replace('.safetensors', '_trainstate.pt')

        with open(config_path, 'w', encoding='utf-8') as f:
            json.dump(config, f, ensure_ascii=False)

        save_model(self, filepath)

        if optimizer is not None:
            torch.save({
                'optimizer': optimizer.state_dict(),
                'iter': iter_step,
                'best_val_loss': best_val_loss
            }, trainstate_path)
            print(f"Model weights and Training state saved to {trainstate_path}")

    @classmethod
    def load_model(cls, filepath, device='cuda'):
        config_path = filepath.replace('.safetensors', '_config.json')
        trainstate_path = filepath.replace('.safetensors', '_trainstate.pt')

        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)

        tokenizer = OptimizedByteLevelBPE(
            merges=[tuple(m) for m in config['tokenizer_config']['merges']],
            vocab=config['tokenizer_config']['vocab'],
            special_tokens=config['tokenizer_config']['special_tokens']
        )

        model = cls(
            vocab_size=config['vocab_size'],
            n_embd=config['n_embd'],
            block_size=config['block_size'],
            n_layer=config['n_layer'],
            n_head=config['n_head'],
            tokenizer=tokenizer,
            device=device
        ).to(device)

        load_model(model, filepath, strict=True)

        train_state = None
        if os.path.exists(trainstate_path):
            train_state = torch.load(trainstate_path, map_location=device)
            print(f"Training state loaded from {trainstate_path}")

        print(f"Model loaded from {filepath} with config {config_path}")
        return model, tokenizer, train_state


In [22]:
# Utility functions for batching
def encode_texts(tokenizer, texts, dropout=0.0):
    """Her metni ayrı tokenize edip liste listesi döner."""
    encoded_texts = []
    for txt in texts:
        token_ids = tokenizer.encode(txt, dropout=dropout)
        encoded_texts.append(token_ids)
    return encoded_texts

def get_batch(data, block_size, batch_size, pad_token_id=None):
    """
    data: 1D tensor, uzun token dizisi
    block_size: model context window
    batch_size: kaç tane örnek alınacak
    """
    # 0..len(data)-block_size-1 arası rastgele başlangıç pozisyonları seç
    max_start_idx = data.size(0) - block_size - 1
    starts = torch.randint(0, max_start_idx, (batch_size,))

    inputs = []
    targets = []
    for start in starts:
        input_seq = data[start : start + block_size]
        target_seq = data[start + 1 : start + block_size + 1]

        # Eğer padding yapılacaksa buraya ekle (genelde 1D tokenlarda gerek yok)
        inputs.append(input_seq.unsqueeze(0))
        targets.append(target_seq.unsqueeze(0))

    xb = torch.cat(inputs, dim=0)   # batch_size x block_size
    yb = torch.cat(targets, dim=0)  # batch_size x block_size
    return xb, yb


@torch.no_grad()
def estimate_loss(model, train_data, val_data, block_size, batch_size, eval_iters, pad_token_id):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(train_data if split == 'train' else val_data, block_size, batch_size, pad_token_id)
            _, loss = model(X.to(device), Y.to(device))
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


def get_lr(it, warmup_iters=500, max_lr=3e-4, min_lr=1e-5, total_iters=10000):
    if it < warmup_iters:
        return max_lr * it / warmup_iters
    elif it > total_iters:
        return min_lr
    else:
        decay_ratio = (it - warmup_iters) / (total_iters - warmup_iters)
        cosine_decay = 0.5 * (1 + math.cos(math.pi * decay_ratio))
        return min_lr + (max_lr - min_lr) * cosine_decay


In [ ]:
def train_model(resume_training=False, checkpoint_path=None):
    
    batch_size = 64
    block_size = 64
    max_iters = 10000
    learning_rate = 3e-4
    eval_interval = 100
    eval_iters = 200
    n_embd = 64
    n_head = 8
    n_layer = 8
    save_interval = 500
    weight_decay = 1e-2
    patience = 3
    accumulation_steps = 4

    full_corpus = load_and_preprocess_data(max_samples=10000)

    tokenizer_path = "tokenizer.json"
    if not os.path.exists(tokenizer_path):
        print("Training tokenizer...")
        tokenizer = OptimizedByteLevelBPE()
        tokenizer.train(full_corpus, num_merges=3000, chunk_size=5000, verbose=True)
        tokenizer.save_model(tokenizer_path)
    else:
        print("Loading pretrained tokenizer...")
        tokenizer = OptimizedByteLevelBPE.load_model(tokenizer_path)

    pad_token_id = tokenizer.token_to_id.get('<pad>', 0)

    def encode_text(text):
        words = re.findall(r'\S+|\s+', text)
        tokens = []
        for word in words:
            tokens.extend(tokenizer.encode(word))
        return tokens

    tokens = []
    for text in tqdm(full_corpus, desc="Encoding texts"):
        tokens.extend(encode_text(text))
    data = torch.tensor(tokens, dtype=torch.long)

    n = int(0.9 * len(data))
    train_data = data[:n]
    val_data = data[n:]

    print(f"Total tokens: {len(data)}")
    print(f"Train data size: {len(train_data)}")
    print(f"Val data size: {len(val_data)}")

    iter_start = 0
    best_val_loss = float('inf')

    if resume_training and checkpoint_path and os.path.exists(checkpoint_path):
        print(f"Resuming training from {checkpoint_path}...")
        model, tokenizer, train_state = Transformer.load_model(checkpoint_path, device=device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        optimizer.load_state_dict(train_state['optimizer'])
        iter_start = train_state['iter']
        best_val_loss = train_state['best_val_loss']
    else:
        vocab_size = len(tokenizer.token_to_id)
        model = Transformer(
            vocab_size=vocab_size,
            n_embd=n_embd,
            block_size=block_size,
            n_layer=n_layer,
            n_head=n_head,
            tokenizer=tokenizer,
            device=device
        ).to(device)
        optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    print(f"{sum(p.numel() for p in model.parameters())/1e6:.2f}M parameters")

    model.train()
    patience_counter = 0
    is_first = True

    scaler = GradScaler(device="cuda")

    for iter in range(iter_start, max_iters):
        lr = get_lr(iter)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        optimizer.zero_grad()
        for acc_step in range(accumulation_steps):
            xb, yb = get_batch(train_data, block_size, batch_size, pad_token_id)
            xb, yb = xb.to(device), yb.to(device)

            with autocast():
                logits, loss = model(xb, yb)
                loss = loss / accumulation_steps

            scaler.scale(loss).backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

        if iter % eval_interval == 0 or iter == max_iters - 1:
            losses = estimate_loss(model, train_data, val_data, block_size, batch_size, eval_iters, pad_token_id)
            train_loss = losses['train']
            val_loss = losses['val']
            print(f"Step {iter}: Train {train_loss:.4f}, Val {val_loss:.4f}, LR {lr:.6f}")

            if not is_first:
                if iter % save_interval == 0:
                    checkpoint_path = f"checkpoints/checkpoint_{iter}.safetensors"
                    model.save_model(checkpoint_path, optimizer=optimizer, iter_step=iter, best_val_loss=best_val_loss)

                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    patience_counter = 0
                    best_model_path = "checkpoints/best_model/best_model.safetensors"
                    model.save_model(best_model_path, optimizer=optimizer, iter_step=iter, best_val_loss=best_val_loss)
                else:
                    patience_counter += 1
                    if patience_counter >= patience:
                        print("Early stopping triggered.")
                        break
            else:
                is_first = False

    return model

In [ ]:
model = train_model()

In [ ]:
model = train_model(resume_training=True, checkpoint_path="checkpoints/best_model/best_model.safetensors")

Preprocessing data: 100%|██████████| 10000/10000 [00:01<00:00, 8948.65it/s]


Loading pretrained tokenizer...


Encoding texts: 100%|██████████| 10000/10000 [01:00<00:00, 164.26it/s]


Total tokens: 1462039
Train data size: 1315835
Val data size: 146204
Resuming training from checkpoints/best_model/best_model.safetensors...


/tmp/ipykernel_62285/204904871.py:180: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_state = torch.load(trainstate_path, map_location=device)


Training state loaded from checkpoints/best_model/best_model_trainstate.pt
Model loaded from checkpoints/best_model/best_model.safetensors with config checkpoints/best_model/best_model_config.json
0.69M parameters


/tmp/ipykernel_62285/3118949519.py:80: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipykernel_62285/3118949519.py:92: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Step 300: Train 6.0685, Val 6.0683, LR 0.000180
Step 400: Train 5.2348, Val 5.2322, LR 0.000240
Model weights and Training state saved to checkpoints/best_model/best_model_trainstate.pt


In [12]:
model_path = "checkpoints/best_model.safetensors"
model = Transformer.load_model(model_path, device)

prompt = "Çin'de yapılan bir araştırmaya göre Çin Seddi'nin yapımı tam 100 yıl sürmüştür ve"

generated = model.generate_from_prompt(
    prompt, 
    max_new_tokens=256, 
    temperature=0.3, 
    top_k=30
)


print("\nGenerated text:")
print(generated)


Generated text:
Çin'de yapılan bir araştırmaya göre Çin Seddi'nin yapımı tam 100 yıl sürmüştür ve daha sonra birçok ülkede kullanılmıştır.The Walter Walter of Fall tarafından geliştirilen ve Kuzey Amerika'da yayınlanan bir video oyunu olan bir video oyunudur ve birçok oyun ve ödül kazanmıştır.Kuzey Amerika'da doğan bir bilim insanıdır ve insan ve doğal olan ve doğal alanlarında uzmanlaşmıştır. Slovak ilinin Çağ ilçesine bağlı olan köy, tarihi Gürcü köyünün adı olan "Kuzey Kuzey Kuzey Kıbrıs Türk Cumhuriyeti'nde yer alan ve Türk halk müziği olup, müzik ve tarihsel eserlerinin de önem


In [ ]:
#Çin'de yapılan bir araştırmaya göre Çin Seddi'nin yapımı tam 100 yıl sürmüştür ve SS S S La tarafından verilen bir olarak kabul edilen bir ve p al olup, da da bu ve ca da bu p ve gibi da da da da da da da da da da da da ve bya bulunur; ayrıca ayrıca da ayrıca da da da da da da da da da da ve da da da da da da da da a da da da a da da da giyda da ve da yer yer almıştır.Dil ve Park Ra Duro Hun S
#Çin'de yapılan bir araştırmaya göre Çin Seddi'nin yapımı tam 100 yıl sürmüştür ve 2014 yılında Panter tarafından yayınlanmıştır.Panz Pear, İngiliz yazar ve akademisyen olup, "Sloven Ke K" adlı eseri olarak bilinir ve "Lin Ke "Lad" adlı eseri "Lad " adlı şarkısıyla tanınır; "Sloven ", "Love " şarkısı" şarkısı, "Soke" ve " " şarkısı " " " şarkısı " sözü" olarak bilinir.Açe Dil tarafından yazılan "The ", "Gece of the " adlı kitabı "Kahraman" kitabı, "Nobel King" ve "Star" adlı eseri ver
#Çin'de yapılan bir araştırmaya göre Çin Seddi'nin yapımı tam 100 yıl sürmüştür ve daha sonra birçok ülkede kullanılmıştır.The Walter Walter of Fall tarafından geliştirilen ve Kuzey Amerika'da yayınlanan bir video oyunu olan bir video oyunudur ve birçok oyun ve ödül kazanmıştır.Kuzey Amerika'da doğan bir bilim insanıdır ve insan ve doğal olan ve doğal alanlarında uzmanlaşmıştır. Slovak ilinin Çağ ilçesine bağlı olan köy, tarihi Gürcü köyünün adı olan "Kuzey Kuzey Kuzey Kıbrıs Türk Cumhuriyeti'nde yer alan ve Türk halk müziği olup, müzik ve tarihsel eserlerinin de önem
